# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [3]:
# Your code here
import pandas as pd
import sqlite3

# Create a connection to the database
conn = sqlite3.connect('data.sqlite')

## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [4]:
# Your code here

query = """SELECT employees.firstName, employees.lastName
FROM employees
LEFT JOIN offices USING(officeCode)
WHERE offices.city = 'Boston' """

df = pd.read_sql_query(query, conn)
print(df)



  firstName   lastName
0     Julie   Firrelli
1     Steve  Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [15]:
# Your code here

query = """
SELECT 
    o.officeCode,
    o.city,
    COUNT(e.employeeNumber) as employee_count
FROM 
    offices o
    LEFT JOIN employees e ON o.officeCode = e.officeCode
GROUP BY 
    o.officeCode, o.city
HAVING 
    COUNT(e.employeeNumber) = 0;
"""
df = pd.read_sql_query(query, conn)
print(df)


   officeCode    city  employee_count
0          27  Boston               0


## Write 3 questions of your own and answer them

In [16]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""
query = """ SELECT offices.officeCode, offices.city, COUNT(customers.customerNumber) AS num_customers
FROM offices
JOIN customers ON customers.customerNumber = customers.salesRepEmployeeNumber
GROUP BY offices.officeCode, offices.city;
 """
df = pd.read_sql_query(query, conn)
print(df)


Empty DataFrame
Columns: [officeCode, city, num_customers]
Index: []


In [19]:
"""
Question 1
"Which customers have never made a payment but have placed orders?"
"""

# Your code here
q = '''SELECT 
    c.customerNumber,
    c.customerName,
    COUNT(o.orderNumber) as number_of_orders
FROM 
    customers c
    LEFT JOIN payments p ON c.customerNumber = p.customerNumber
    JOIN orders o ON c.customerNumber = o.customerNumber
WHERE 
    p.checkNumber IS NULL
GROUP BY 
    c.customerNumber, c.customerName'''

df = pd.read_sql_query(q, conn)
print(df)

Empty DataFrame
Columns: [customerNumber, customerName, number_of_orders]
Index: []


In [20]:
"""
Question 2
Who are the top 5 sales representatives based on their customers' total order value?"
"""

# Your code here
q = '''SELECT 
    e.employeeNumber,
    e.firstName,
    e.lastName,
    COUNT(DISTINCT c.customerNumber) as number_of_customers,
    SUM(od.quantityOrdered * od.priceEach) as total_sales
FROM 
    employees e
    JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
    JOIN orders o ON c.customerNumber = o.customerNumber
    JOIN orderdetails od ON o.orderNumber = od.orderNumber
GROUP BY 
    e.employeeNumber, e.firstName, e.lastName
ORDER BY 
    total_sales DESC
LIMIT 5'''

df = pd.read_sql_query(q, conn)
print(df)


   employeeNumber firstName   lastName  number_of_customers  total_sales
0            1370    Gerard  Hernandez                    7   1258577.81
1            1165    Leslie   Jennings                    6   1081530.54
2            1401    Pamela   Castillo                   10    868220.55
3            1501     Larry       Bott                    8    732096.79
4            1504     Barry      Jones                    9    704853.91


In [21]:
"""
Question 3
Which product lines have the highest profit margin (MSRP vs buyPrice)?
"""

# Your code here
q = """SELECT 
    pl.productLine,
    COUNT(p.productCode) as number_of_products,
    AVG((p.MSRP - p.buyPrice) / p.buyPrice * 100) as avg_profit_margin_percentage
FROM 
    productlines pl
    JOIN products p ON pl.productLine = p.productLine
GROUP BY 
    pl.productLine
ORDER BY 
    avg_profit_margin_percentage DESC"""

df = pd.read_sql_query(q, conn)
print(df)

        productLine  number_of_products  avg_profit_margin_percentage
0       Motorcycles                  13                     93.795235
1      Vintage Cars                  24                     90.703168
2  Trucks and Buses                  11                     90.534061
3             Ships                   9                     88.586586
4      Classic Cars                  38                     87.647237
5            Planes                  12                     83.268776
6            Trains                   3                     79.358266


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [22]:
# Your code here
q = """SELECT 
    e.firstName,
    e.lastName,
    p.productName
FROM 
    employees e
    JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
    JOIN orders o ON c.customerNumber = o.customerNumber
    JOIN orderdetails od ON o.orderNumber = od.orderNumber
    JOIN products p ON od.productCode = p.productCode
ORDER BY 
    e.lastName, e.firstName, p.productName"""

df = pd.read_sql_query(q, conn)
print(df)

     firstName lastName                productName
0         Loui   Bondur      18th century schooner
1         Loui   Bondur     1900s Vintage Bi-Plane
2         Loui   Bondur    1900s Vintage Tri-Plane
3         Loui   Bondur          1903 Ford Model A
4         Loui   Bondur          1903 Ford Model A
...        ...      ...                        ...
2991    George   Vanauf                The Titanic
2992    George   Vanauf                The Titanic
2993    George   Vanauf  The USS Constitution Ship
2994    George   Vanauf  The USS Constitution Ship
2995    George   Vanauf  The USS Constitution Ship

[2996 rows x 3 columns]


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [23]:
# Your code here
q = """SELECT 
    e.lastName,
    e.firstName,
    SUM(od.quantityOrdered) as total_products_sold
FROM 
    employees e
    JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
    JOIN orders o ON c.customerNumber = o.customerNumber
    JOIN orderdetails od ON o.orderNumber = od.orderNumber
GROUP BY 
    e.lastName, e.firstName
ORDER BY 
    e.lastName, e.firstName"""

df = pd.read_sql_query(q, conn)
print(df)

     lastName firstName  total_products_sold
0      Bondur      Loui                 6186
1        Bott     Larry                 8205
2    Castillo    Pamela                 9290
3    Firrelli     Julie                 4227
4      Fixter      Andy                 6246
5      Gerard    Martin                 4180
6   Hernandez    Gerard                14231
7    Jennings    Leslie                11854
8       Jones     Barry                 7486
9       Marsh     Peter                 6632
10      Nishi      Mami                 4923
11  Patterson     Steve                 5561
12   Thompson    Leslie                 4056
13      Tseng  Foon Yue                 5016
14     Vanauf    George                 7423


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [24]:
# Your code here
q = """SELECT 
    e.firstName,
    e.lastName,
    COUNT(DISTINCT od.productCode) as unique_products_sold
FROM 
    employees e
    JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
    JOIN orders o ON c.customerNumber = o.customerNumber
    JOIN orderdetails od ON o.orderNumber = od.orderNumber
GROUP BY 
    e.firstName, e.lastName
HAVING 
    COUNT(DISTINCT od.productCode) > 200
ORDER BY 
    unique_products_sold DESC"""

df = pd.read_sql_query(q, conn)
print(df)

Empty DataFrame
Columns: [firstName, lastName, unique_products_sold]
Index: []


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!